# Phase 2: Model Training and Experimentation

This notebook will focus on training and optimizing our model. Our objectives include
1. Experiment tracking with MLflow
2. Cross-validation
3. Hyperparameter optimization
4. Model evaluation and selection
5. Model versioning and registration

In [1]:
import mlflow
# ignore this: required for run_notebooks.sh
%pip install --upgrade pip --quiet
%pip install mlflow optuna --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import optuna
import xgboost as xgb

TRACKING_URI = "sqlite:///../mlflow.db" #"./mlruns"

print("Loading data...")
import ames_notebooks
from app.data_ingestion.read_data import DataReader

reader = DataReader()
train_data, test_data = reader.load_train_test()
print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)

Loading data...
Train shape: (1460, 80)
Test shape: (1459, 79)


In [3]:
# get preprocessing pipeline
from app.pipelines.preprocessing import get_fitted_pipelines
feature_preprocessor, target_transformer = get_fitted_pipelines(train_data)

In [4]:
X = train_data.drop('SalePrice', axis=1)
y = train_data['SalePrice']

# apply pipelines/transformations
X_processed = feature_preprocessor.transform(X)
y_processed = target_transformer.transform(y)

# Split data into train and validation sets
# use _val to prevent confusion between the test dataset
X_train, X_val, y_train, y_val = train_test_split(
    X_processed, y_processed, 
    test_size=0.2, 
    random_state=42
)

# Apply preprocessing
print("Applying preprocessing pipeline...")
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)

Applying preprocessing pipeline...
X_train shape: (1168, 241)
X_val shape: (292, 241)


## Baseline Model Development

Let's start with a simple baseline model using XGBoost with default parameters. This will give us a reference point for further improvements.

In [5]:
from app.pipelines.training import XGBModelTrainer

model = xgb.XGBRegressor(random_state=42, n_jobs=-1)

trainer = XGBModelTrainer(tracking_uri=TRACKING_URI)
baseline_model = trainer.train(model, "xgboost-baseline", X_train, y_train, X_val, y_val)

print("\nTraining Metrics:")
for metric, value in trainer.train_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nValidation Metrics:")
for metric, value in trainer.val_metrics.items():
    print(f"{metric}: {value:.4f}")

2025/11/20 09:05:58 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/11/20 09:05:58 INFO mlflow.store.db.utils: Updating database tables
2025-11-20 09:05:58 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-11-20 09:05:58 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025-11-20 09:05:58 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-11-20 09:05:58 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


[0]	validation_0-rmse:0.33887
[99]	validation_0-rmse:0.14973


/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1115: UserWarning: [09:05:58] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)
/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1124: UserWarning: [09:06:02] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  self.get_booster().load_model(fname)
2025/11/20 09:06:02 INFO mlflow.models.model: Found the following environment variables used during model inference: [API_KEY, STRIPE_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
2025/11/20 09:06:02 INFO mlflow.store.db.utils: Crea


Training Metrics:
train_rmse: 0.0078
train_mae: 0.0054
train_r2: 0.9996

Validation Metrics:
val_rmse: 0.1497
val_mae: 0.1038
val_r2: 0.8799


Registered model 'xgboost-baseline' already exists. Creating a new version of this model...
Created version '7' of model 'xgboost-baseline'.


## Hyperparameter Optimization with Optuna

Now that we have a baseline model, let's use Optuna to find better hyperparameters for our XGBoost model. We'll define an objective function that Optuna will optimize using cross-validation scores.

In [6]:
# objective function for Optuna
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'random_state': 42
    }
    
    # get model with suggested parameters
    model = xgb.XGBRegressor(**params)
    
    # cross-validation
    cv_scores = cross_val_score(
        model, 
        X_train, 
        y_train, 
        cv=5, 
        scoring='neg_root_mean_squared_error',
        n_jobs=-1
    )
    
    # mean negative RMSE (Optuna minimizes objective)
    return -cv_scores.mean()

# Create and run Optuna study
study = optuna.create_study(direction='minimize')
optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2025-11-20 09:06:02,381] A new study created in memory with name: no-name-4b77d699-65c3-4992-a0ad-c3a7b4a81a27


  0%|          | 0/50 [00:00<?, ?it/s]

Best trial:
  Value:  0.12411194491876767
  Params: 
    max_depth: 4
    learning_rate: 0.03167072557201432
    n_estimators: 780
    min_child_weight: 7
    subsample: 0.8376965123964913
    colsample_bytree: 0.6328065474038497
    reg_alpha: 9.303927966864939e-05
    reg_lambda: 2.9200070482148033e-05


In [7]:
best_params = study.best_params
best_params['random_state'] = 42
model = xgb.XGBRegressor(**best_params)

optimized_model = trainer.train(model, "xgboost-optimized", X_train, y_train, X_val, y_val)

print("\nTraining Metrics:")
for metric, value in trainer.train_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nValidation Metrics:")
for metric, value in trainer.val_metrics.items():
    print(f"{metric}: {value:.4f}")

[0]	validation_0-rmse:0.42263
[100]	validation_0-rmse:0.15083
[200]	validation_0-rmse:0.13869
[300]	validation_0-rmse:0.13590
[400]	validation_0-rmse:0.13493
[500]	validation_0-rmse:0.13451
[600]	validation_0-rmse:0.13398
[700]	validation_0-rmse:0.13404
[779]	validation_0-rmse:0.13403


/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1115: UserWarning: [09:06:34] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)
/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1124: UserWarning: [09:06:37] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  self.get_booster().load_model(fname)
Registered model 'xgboost-optimized' already exists. Creating a new version of this model...
Created version '6' of model 'xgboost-optimized'.



Training Metrics:
train_rmse: 0.0442
train_mae: 0.0317
train_r2: 0.9872

Validation Metrics:
val_rmse: 0.1340
val_mae: 0.0884
val_r2: 0.9037


## Model Comparison

Let's compare the performance of our baseline and optimized models to see the improvement from hyperparameter optimization.

In [8]:
from app.pipelines.training import evaluate_model

# compare models on validation set
baseline_metrics = evaluate_model(baseline_model, X_val, y_val)
optimized_metrics = evaluate_model(optimized_model, X_val, y_val)

print("Baseline Model Metrics:")
for metric, value in baseline_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nOptimized Model Metrics:")
for metric, value in optimized_metrics.items():
    print(f"{metric}: {value:.4f}")

improvement = (baseline_metrics['rmse'] - optimized_metrics['rmse']) / baseline_metrics['rmse'] * 100
print(f"\nRMSE Improvement: {improvement:.2f}%")

Baseline Model Metrics:
rmse: 0.1497
mae: 0.1038
r2: 0.8799

Optimized Model Metrics:
rmse: 0.1340
mae: 0.0884
r2: 0.9037

RMSE Improvement: 10.48%


In [22]:
from app.pipelines.preprocessing import get_fitted_pipelines
feature_preprocessor, target_transformer = get_fitted_pipelines(train_data)

from app.inference.predict import AmesPredictor
predictor = AmesPredictor(feature_engineer=feature_preprocessor, tracking_uri=TRACKING_URI, model_name="xgboost-optimized")
predictor.model

/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1124: UserWarning: [09:12:08] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  self.get_booster().load_model(fname)


mlflow.pyfunc.loaded_model:
  artifact_path: /Users/nic/git/AmesHousingPredictor/notebooks/mlruns/1/models/m-a81e284cb6ae4d88aef2876b59d6b3a3/artifacts
  flavor: mlflow.xgboost
  run_id: efa3c97342fc4878b55faebee6bbf5cd

In [23]:
# Example: predict on a single row from the test set (keeps original columns)
example_row = test_data.iloc[[0]]  # DataFrame with one row
scaled_prediction = predictor.predict(example_row)
prediction = target_transformer.inverse_transform(scaled_prediction)
print("Example prediction (SalePrice):", prediction)

Example prediction (SalePrice): [121696.336]
